# TESTING AN ONLINE CRYPTO TRADER ADVISOR WITH
- email sending advice
- saving data
- connecting to cex.io and collecting data in real time

In [1]:
from py_files.turtle_buy_strat import *
from py_files.cex_api_wrapper import *

In [2]:
import json
with open('json_files//data.json', 'r') as fp:
        parameters_dict = json.load(fp)
with open('json_files//BTCUSD.json', 'r') as fp:
        dictioBTCUSD = json.load(fp)
with open('json_files//BCHUSD.json', 'r') as fp:
        dictioBCHUSD = json.load(fp)  
with open('json_files//ETHUSD.json', 'r') as fp:
        dictioETHUSD = json.load(fp)
with open('json_files//ETHBTC.json', 'r') as fp:
        dictioETHBTC = json.load(fp)  
with open('json_files//LTCUSD.json', 'r') as fp:
        dictioLTCUSD = json.load(fp)
with open('json_files//XRPUSD.json', 'r') as fp:
        dictioXRPUSD = json.load(fp)  

In [3]:
import time
import pandas as pd
import numpy as np

In [4]:
BTCUSD= pd.read_csv('csv_files//Bitstamp_BTCUSD_1h.csv', skiprows=1, parse_dates=['date'])
BCHUSD= pd.read_csv('csv_files//Bitstamp_BCHUSD_1h.csv', skiprows=1, parse_dates=['date'])
ETHBTC= pd.read_csv('csv_files//Bitstamp_ETHBTC_1h.csv', skiprows=1, parse_dates=['date'])
ETHUSD= pd.read_csv('csv_files//Bitstamp_ETHUSD_1h.csv', skiprows=1, parse_dates=['date'])
LTCUSD= pd.read_csv('csv_files//Bitstamp_LTCUSD_1h.csv', skiprows=1, parse_dates=['date'])
XRPUSD= pd.read_csv('csv_files//Bitstamp_XRPUSD_1h.csv', skiprows=1, parse_dates=['date'])
dfs=[BTCUSD,BCHUSD,ETHBTC,ETHUSD,LTCUSD,XRPUSD] 

In [5]:
TestTurtleBTCUSD=TurtleBuyOnlyOnlineStrategy(dictioBTCUSD)
TestTurtleETHUSD=TurtleBuyOnlyOnlineStrategy(dictioETHUSD)
TestTurtleETHBTC=TurtleBuyOnlyOnlineStrategy(dictioETHBTC)
TestTurtleLTCUSD=TurtleBuyOnlyOnlineStrategy(dictioLTCUSD)
TestTurtleBCHUSD=TurtleBuyOnlyOnlineStrategy(dictioBCHUSD)
TestTurtleXRPUSD=TurtleBuyOnlyOnlineStrategy(dictioXRPUSD)

In [8]:
stop=int(TestTurtleBTCUSD.delta1/TestTurtleBTCUSD.delta2)*TestTurtleBTCUSD.window_delta1

TestTurtleBTCUSD.initial_feeder(list(BTCUSD.open.values[:stop][::-1]))
TestTurtleETHBTC.initial_feeder(list(ETHBTC.open.values[:stop][::-1]))
TestTurtleETHUSD.initial_feeder(list(ETHUSD.open.values[:stop][::-1]))
TestTurtleLTCUSD.initial_feeder(list(LTCUSD.open.values[:stop][::-1]))
TestTurtleBCHUSD.initial_feeder(list(BCHUSD.open.values[:stop][::-1]))
TestTurtleXRPUSD.initial_feeder(list(XRPUSD.open.values[:stop][::-1]))

In [9]:
username=parameters_dict['CEX_username']
api_key=parameters_dict['CEX_api_key']
api_secret=parameters_dict['CEX_api_secret']
api = Api(username, api_key, api_secret)

online_df_saverBTCUSD=cex_dataframe_online_saver('BTCUSD_price')
online_df_saverBCHUSD=cex_dataframe_online_saver('BCHUSD_price')
online_df_saverETHBTC=cex_dataframe_online_saver('ETHBTC_price')
online_df_saverETHUSD=cex_dataframe_online_saver('ETHUSD_price')
online_df_saverLTCUSD=cex_dataframe_online_saver('LTCUSD_price')
online_df_saverXRPUSD=cex_dataframe_online_saver('XRPUSD_price')


In [ ]:
import time
import datetime as dt 

savers=[ online_df_saverBTCUSD,online_df_saverBCHUSD,online_df_saverETHBTC,
      online_df_saverETHUSD,online_df_saverLTCUSD,online_df_saverXRPUSD]
symbols=['BTC/USD','BCH/USD','ETH/BTC','ETH/USD','LTC/USD','XRP/USD']
turtles=[
    TestTurtleBTCUSD,TestTurtleBCHUSD ,TestTurtleETHBTC ,
    TestTurtleETHUSD ,TestTurtleLTCUSD ,TestTurtleXRPUSD]

while True:
    
    t = dt.datetime.now()   
    if (t.minute==0) : 
        for save,sym,turtle in  zip(savers,symbols,turtles):
            
            tick=api.ticker(sym)
            save.cex_ticker_save(tick)
            feeder={'timestamp':pd.to_datetime( tick['timestamp'][0], unit='s')
                    ,'bid':tick['bid'][0],'ask':tick['ask'][0]}
            turtle.run(feeder)

        time.sleep(int(TestTurtleBTCUSD.delta2*1.1))
        